根据得到的正负样本去训练分类器并分析所有选手

In [16]:
# c:\Users\Administrator\AppData\Local\Programs\Python\Python312\python.exe -m pip install tensorflow
import sys
print(sys.executable)
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
positive_df['label']=1
negative_df['label']=0
train_full=pd.concat([positive_df,negative_df],axis=0,ignore_index=True)

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\python.exe


In [17]:
#dynamic
positive_df=pd.read_csv('Dynamic_1_final.csv')
negative_df=pd.read_csv('Dynamic_0_final.csv')
positive_df.columns

Index(['Unnamed: 0', 'CenterControlScore', 'PieceActivityScore',
       'KingSafetyScore', 'CastlingScore', 'KingTropismScore',
       'KingDefendersScore', 'KingPawnShieldScore', 'KingZoneControlScore',
       'KingDiagonalExposureScore', 'KingEscapeSquaresScore', 'CaptureRatio',
       'PawnCenter', 'AveragePawnAdvanceDepth', 'CheckRatio', 'ForceRatio',
       'PawnIsolateScore', 'PawnOverlapScore', 'PawnProtectScore',
       'queen_1LifeRatio', 'rook_1LifeRatio', 'rook_2LifeRatio',
       'bishop_1LifeRatio', 'bishop_2LifeRatio', 'knight_1LifeRatio',
       'knight_2LifeRatio', 'pawn_1LifeRatio', 'pawn_2LifeRatio',
       'pawn_3LifeRatio', 'pawn_4LifeRatio', 'pawn_5LifeRatio',
       'pawn_6LifeRatio', 'pawn_7LifeRatio', 'pawn_8LifeRatio',
       'queen_promo_1LifeRatio', 'queen_promo_2LifeRatio',
       'queen_promo_3LifeRatio', 'queen_promo_4LifeRatio',
       'queen_promo_5LifeRatio', 'queen_promo_6LifeRatio',
       'queen_promo_7LifeRatio', 'queen_promo_8LifeRatio', 'Name', 'E

In [19]:
#特征列
ignore_cols = ['Unnamed: 0', 'Name', 'ELO', 'label']
feature_cols = [col for col in train_full.columns if col not in ignore_cols]
for col in feature_cols:
    mean_val = train_full[col].mean()
    train_full[col] = train_full[col].fillna(mean_val)
print("缺失值填充完毕！再次检查 NaN:", train_full[feature_cols].isnull().values.any())

# 准备全量的 X 和 y
X_full = train_full[feature_cols].values
y_full = train_full['label'].values

print("划分验证集，找最佳epochs")
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

#标准化
scaler_split = StandardScaler()
X_train_split_scaled = scaler_split.fit_transform(X_train_split)
X_val_split_scaled = scaler_split.transform(X_val_split)

#模型构建函数
def build_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2)) 
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
model_scout = build_model(X_train_split.shape[1])

#Early Stopping
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True,
    verbose=1
)

#找拿到 best_epoch
history = model_scout.fit(
    X_train_split_scaled, y_train_split,
    validation_data=(X_val_split_scaled, y_val_split),
    epochs=100, #较大的上限，反正早停会拦住
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 获取最佳轮数
# 如果早停触发了，best_epoch 就是停止时的轮数减去 patience；如果没有触发，就是设置的 epochs
# 实际上 EarlyStopping 比较智能，我们可以直接看它实际跑了多少轮
optimal_epochs = len(history.history['loss']) 
if early_stop.stopped_epoch > 0:
    # stopped_epoch 是索引（从0开始），且包含了耐心值的轮数
    # 为了保险，我们取最佳权重的那个 epoch 数量
    # (patience 期间 loss 没降，所以最佳 epoch 是 stopped_epoch - patience + 1)
    optimal_epochs = max(1, early_stop.stopped_epoch - 10 + 1)

print(f"\n最佳训练轮数约是: {optimal_epochs} 轮\n")


缺失值填充完毕！再次检查 NaN: False
划分验证集，找最佳epochs
Epoch 1/100


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6167 - loss: 0.6586 - val_accuracy: 0.8140 - val_loss: 0.5886
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8099 - loss: 0.5324 - val_accuracy: 0.8430 - val_loss: 0.4569
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8492 - loss: 0.4234 - val_accuracy: 0.8430 - val_loss: 0.3738
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8616 - loss: 0.3567 - val_accuracy: 0.8471 - val_loss: 0.3433
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8771 - loss: 0.3215 - val_accuracy: 0.8430 - val_loss: 0.3407
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8988 - loss: 0.2859 - val_accuracy: 0.8430 - val_loss: 0.3406
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8905 - loss: 0.2829 - val_accuracy: 0.8430 - val_loss: 0.3460
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8988 - loss: 0.2651 - val_accuracy: 0.8471 - val_loss: 0.3

In [ ]:
#特征列
ignore_cols = ['Unnamed: 0', 'Name', 'ELO', 'label']
feature_cols = [col for col in train_full.columns if col not in ignore_cols]
# 准备全量的 X 和 y
X_full = train_full[feature_cols].values
y_full = train_full['label'].values

print("划分验证集，找最佳epochs")
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

#标准化
scaler_split = StandardScaler()
X_train_split_scaled = scaler_split.fit_transform(X_train_split)
X_val_split_scaled = scaler_split.transform(X_val_split)

#模型构建函数
def build_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2)) 
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
model_scout = build_model(X_train_split.shape[1])

#Early Stopping
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True,
    verbose=1
)

#找拿到 best_epoch
history = model_scout.fit(
    X_train_split_scaled, y_train_split,
    validation_data=(X_val_split_scaled, y_val_split),
    epochs=100, #较大的上限，反正早停会拦住
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 获取最佳轮数
# 如果早停触发了，best_epoch 就是停止时的轮数减去 patience；如果没有触发，就是设置的 epochs
# 实际上 EarlyStopping 比较智能，我们可以直接看它实际跑了多少轮
optimal_epochs = len(history.history['loss']) 
if early_stop.stopped_epoch > 0:
    # stopped_epoch 是索引（从0开始），且包含了耐心值的轮数
    # 为了保险，我们取最佳权重的那个 epoch 数量
    # (patience 期间 loss 没降，所以最佳 epoch 是 stopped_epoch - patience + 1)
    optimal_epochs = max(1, early_stop.stopped_epoch - 10 + 1)

print(f"\n最佳训练轮数约是: {optimal_epochs} 轮\n")


# ==========================================
# 第二阶段：全量实战 (Retrain on Full Data)
# ==========================================
print(f"用全量数据训练 {optimal_epochs} 轮 ---")

# 1. 全量标准化 (这次根据所有数据拟合 scaler)
scaler_final = StandardScaler()
X_full_scaled = scaler_final.fit_transform(X_full)

# 2. 重新构建一个干净的模型
model_final = build_model(X_full.shape[1])

# 3. 全量训练 (不再需要 validation_data，也不需要 early_stop，直接跑最佳轮数)
model_final.fit(
    X_full_scaled, y_full,
    epochs=optimal_epochs, 
    batch_size=32,
    verbose=1
)

# ==========================================
# 第三阶段：预测 Test Data
# ==========================================
print("\n--- 第三阶段：预测 Test Data ---")

# 1. 准备测试数据
X_test = test_df[feature_cols].values

# 2. 使用全量的 scaler 进行转换 (必须用 scaler_final)
X_test_scaled = scaler_final.transform(X_test)

# 3. 预测概率
probs = model_final.predict(X_test_scaled).flatten()

# 4. 写入结果
new_col_name = 'Style_Prob_NN'
test_df[new_col_name] = probs

print(f"预测完成！结果已保存至列: {new_col_name}")
print(test_df[['Name', new_col_name]].head())

# test_df.to_csv("final_result.csv", index=False)